In [16]:
from firebase_admin import credentials, firestore
from datetime import datetime
import firebase_admin
import logging
import json
import os

In [17]:
crawled_telegram_data_path = "../data/crawled_telegram_data.json"
gemini_result_data_path = "../data/gemini_result_data.json"

with open(crawled_telegram_data_path, "r") as f:
    crawled_telegram_data = json.load(f)

with open(gemini_result_data_path,"r") as f:
    gemini_result_data = json.load(f)

In [18]:
firebase_key_path = '../../data/firebase-key.json'

if not firebase_admin._apps:
    cred = credentials.Certificate(firebase_key_path)
    firebase_admin.initialize_app(cred)
db = firestore.client()


In [19]:
category = "analyzed_telegram_data"

In [20]:
# 로그 디렉토리 생성
log_dir = "../logs"
os.makedirs(log_dir, exist_ok=True)

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f"{log_dir}/mongodb_operations.log", encoding='utf-8'),
        logging.StreamHandler()  # 콘솔에도 출력
    ]
)

logger = logging.getLogger("mongodb_operations")

In [21]:
# 섹터 데이터 저장
def save_sector_data(sector, date_str, data):
    try:
        doc_ref = db.collection(category).document(sector)
        doc = doc_ref.get()
        
        # 데이터에 원본 날짜 문자열도 포함
        data["date_str"] = date_str
        
        if doc.exists:
            # 기존 데이터가 있으면 가져오기
            existing_data = doc.to_dict() or {}
            # 날짜 키는 문자열로 사용하고, 데이터 안에 timestamp 필드 추가
            if "dates" not in existing_data:
                existing_data["dates"] = {}
            
            # 날짜별 데이터 저장 (문자열 키 사용)
            existing_data["dates"][date_str] = data
            # timestamp 필드 추가
            data["timestamp"] = date_str
            
            # 전체 데이터 저장
            doc_ref.set(existing_data)
        else:
            # 문서가 없으면 새로 생성
            # timestamp 필드 추가
            data["timestamp"] = date_str
            doc_ref.set({"dates": {date_str: data}})
            
        logger.info(f"{sector} {date_str} 데이터 저장 완료")
        return f"{sector} {date_str} 데이터 저장 완료"
    except Exception as e:
        error_msg = f"{sector} {date_str} 데이터 저장 중 오류 발생: {e}"
        logger.error(error_msg)
        return error_msg


In [ ]:
for sector in crawled_telegram_data.keys():
    if sector == '':
        pass
    else:
        data = {"date":"", "channels":[], "summary":"", "headline":""}
        for date in crawled_telegram_data[sector]:
            data["date"] = date
            for channel in crawled_telegram_data[sector][date]:
                analysis_channel = {
                    "channel_name": channel, 
                    "posts": crawled_telegram_data[sector][date][channel]["posts"],
                    "subscribers": crawled_telegram_data[sector][date][channel]["subscribers"],
                }

                try:
                    # None 값을 안전하게 처리하기 위해 or 연산자 사용
                    sector_data = gemini_result_data.get(sector, {}) or {}
                    date_data = sector_data.get(date, {}) or {}
                    scores = date_data.get("scores", {}) or {}
                    
                    analysis_channel["score"] = scores.get(channel, 50)
                except Exception as e:
                    print(f"점수 가져오기 오류 ({sector}/{date}/{channel}): {e}")
                    analysis_channel["score"] = -1
                
                data["channels"].append(analysis_channel)
            
            # 요약 정보 처리 (None 값 처리 추가)
            try:
                # None 값을 안전하게 처리하기 위해 or 연산자 사용
                sector_data = gemini_result_data.get(sector, {}) or {}
                date_data = sector_data.get(date, {}) or {}
                summary_data = date_data.get("summary", "") or ""
                
                # 타입 체크를 통해 다른 형식 처리
                if isinstance(summary_data, dict):
                    # 딕셔너리 형식일 경우
                    data["headline"] = summary_data.get("headline", "")
                    data["summary"] = summary_data.get("summary", "")
                elif isinstance(summary_data, str):
                    # 문자열 형식일 경우
                    data["summary"] = summary_data
                    data["headline"] = ""  # headline은 없음
                else:
                    # 다른 형식이면 빈 값으로 설정
                    data["summary"] = ""
                    data["headline"] = ""
                    
            except Exception as e:
                print(f"요약 가져오기 오류 ({sector}/{date}): {e}")
                data["summary"] = ""
                data["headline"] = ""
                
            save_sector_data(sector, date, data)